In [ ]:
import os
os.chdir('./..')

import sys
import time
import concurrent.futures
import pickle

import Methods.common as common
import Methods.dataLoader as dataloader

try:
    import argparse
    import magic
except ImportError as err:
    print (err)
    sys.exit(-1)

In [ ]:
dataset_file = 'sorted_all_dataset.pkl'
token_file = 'tokens.txt'

In [ ]:
with open(dataset_file, "rb") as f:
    dataset = pickle.load(f)

token_list = list()
with open(token_file) as f:
    lines = f.readlines()
    for line in lines[0].split(','):
        token_list.append(line)
        
lenTokens = len(token_list)

try:
    common.magic_cookie = magic.open(magic.MAGIC_MIME)
    common.magic_cookie.load()
except AttributeError:
    common.magic_cookie = magic.Magic(mime=True, uncompress=True)
common.verbose_print('[-] initialized magic cookie\n')

In [ ]:
def getDataUsingThreads(params):
    source, destination, prs, cut_off_date, task, begin, end = params
    start = time.perf_counter()
    
    ct = 0
    
    working_repos = []
    for pair in range(begin, end):
        print(pair, '-', source[pair], '-', destination[pair])
        common.verbose_mode = False

        try:
            destination_sha, ct = dataloader.getDestinationSha(destination[pair], cut_off_date[pair], token_list, ct)
            ct, pr_data, req, runtime = dataloader.fetchPrData(source[pair], destination[pair], prs[pair], destination_sha, token_list, ct)
            with open('Repos_prs/' + str(pair) + '_' + source[pair].split('/')[0] + '_' + source[pair].split('/')[1] + '_prs.pkl', 'wb') as f:
                pickle.dump([pr_data, req, runtime], f)
            working_repos.append(pair)
        except Exception as e:
            print('Exception  :(\n ',e)
    print(working_repos)        
    finish = time.perf_counter()
    return f'Task {task} done executing in ... {round((finish - start) / 60, 2)} minutes'

def mainThread():
    source = []
    destination = []
    prs = []
    cut_off_date = []
    
    for i in dataset:
        source.append(dataset[i]['source'])
        destination.append(dataset[i]['destination'])
        prs.append(str(dataset[i]['pr']).split('/'))
        cut_off_date.append(dataset[i]['cut_off_date'])
        
    task = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    
    start1 = 1
    end1 = 2
    start2 = 4
    end2 = 6
    start3 = 4
    end3 = 6
    start4 = 7
    end4 = 10
    start5 = 10
    end5 = 15
    start6 = 18
    end6 = 19
    start7 = 21
    end7 = 31
    start8 = 33
    end8 = 45
    start9 = 48
    end9 = 50
    start10 = 150
    end10 = 342
#     start10 = 301
#     end10 = 321
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        params = [
                  (source, destination, prs, cut_off_date, task[0], start1, end1),
                  (source, destination, prs, cut_off_date, task[1], start2, end2),
                  (source, destination, prs, cut_off_date, task[2], start3, end3),
                  (source, destination, prs, cut_off_date, task[3], start4, end4),
                  (source, destination, prs, cut_off_date, task[4], start5, end5),
                  (source, destination, prs, cut_off_date, task[5], start6, end6),
                  (source, destination, prs, cut_off_date, task[6], start7, end7),
                  (source, destination, prs, cut_off_date, task[7], start8, end8),
                  (source, destination, prs, cut_off_date, task[8], start9, end9),
                  (source, destination, prs, cut_off_date, task[9], start10, end10)
                 ]
        results = executor.map(getDataUsingThreads, params)

        for result in results:
            print(result)

mainThread()